In [1]:
import pandas as pd
import xgboost

monthly_payments = pd.read_csv('data/monthly_payments.csv')
monthly_payments['date']=pd.to_datetime(monthly_payments['date'])
user_table = pd.read_csv('data/user_table.csv')

In [2]:
user_table

,user_id,f_0,f_1,f_2,f_3,f_4
0,117,0.185834,0.257087,0.681947,0.343844,0.562220
1,140,0.274930,0.042934,0.627473,0.534509,0.493101
2,159,0.548740,0.605102,0.369127,0.384560,0.220441
3,175,0.308355,0.600812,0.556980,0.167359,0.453548
4,297,0.724011,0.186172,0.323603,0.579623,0.021604
...,...,...,...,...,...,...
49995,49585,0.691765,0.368827,0.268830,0.536567,0.158915
49996,49751,0.569852,0.289845,0.461525,0.374787,0.487635
49997,49782,0.540032,0.502296,0.185834,0.075061,0.644900
49998,49908,0.420014,0.449307,0.200692,0.535565,0.542775


In [3]:
monthly_payments

,user_id,date,payment
0,73,2017-10-01 00:00:00+00:00,1
1,216,2017-10-01 00:00:00+00:00,1
2,3586,2017-10-01 00:00:00+00:00,1
3,4197,2017-10-01 00:00:00+00:00,1
4,4680,2017-10-01 00:00:00+00:00,1
...,...,...,...
198110,47915,2017-12-01 00:00:00+00:00,1
198111,48003,2017-12-01 00:00:00+00:00,1
198112,48774,2017-12-01 00:00:00+00:00,1
198113,49814,2017-12-01 00:00:00+00:00,1


In [4]:
monthly_payments.loc[lambda row: row['user_id'] == 117].sort_values('date')

,user_id,date,payment
49624,117,2016-08-01 00:00:00+00:00,1
179485,117,2016-09-01 00:00:00+00:00,1
30483,117,2016-10-01 00:00:00+00:00,1
163337,117,2016-11-01 00:00:00+00:00,1
5412,117,2016-12-01 00:00:00+00:00,1
125695,117,2017-01-01 00:00:00+00:00,1
61818,117,2017-02-01 00:00:00+00:00,1
147600,117,2017-03-01 00:00:00+00:00,1
72659,117,2017-04-01 00:00:00+00:00,1
116737,117,2017-05-01 00:00:00+00:00,1


In [5]:
first_observed_date = monthly_payments['date'].min()
last_observed_date = monthly_payments['date'].max()
first_observed_date, last_observed_date

(Timestamp('2016-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2018-01-01 00:00:00+0000', tz='UTC'))

In [6]:
from helper_functions import feature_generation

df = feature_generation(monthly_payments=monthly_payments, user_table=user_table)

In [7]:
df

f_0       f_1       f_2       f_3  \
user_id date                                                                
117     2016-08-01 00:00:00+00:00  0.185834  0.257087  0.681947  0.343844   
        2016-09-01 00:00:00+00:00  0.185834  0.257087  0.681947  0.343844   
        2016-10-01 00:00:00+00:00  0.185834  0.257087  0.681947  0.343844   
        2016-11-01 00:00:00+00:00  0.185834  0.257087  0.681947  0.343844   
        2016-12-01 00:00:00+00:00  0.185834  0.257087  0.681947  0.343844   
...                                     ...       ...       ...       ...   
49751   2016-04-01 00:00:00+00:00  0.569852  0.289845  0.461525  0.374787   
        2016-05-01 00:00:00+00:00  0.569852  0.289845  0.461525  0.374787   
49782   2016-08-01 00:00:00+00:00  0.540032  0.502296  0.185834  0.075061   
49908   2016-03-01 00:00:00+00:00  0.420014  0.449307  0.200692  0.535565   
49934   2016-05-01 00:00:00+00:00  0.636215  0.440063  0.393347  0.496451   

                                        f_4  churned_3m  
user_id date                                             
117     2016-08-01 00:00:00+00:00  0.562220       False  
        2016-09-01 00:00:00+00:00  0.562220       False  
        2016-10-01 00:00:00+00:00  0.562220       False  
        2016-11-01 00:00:00+00:00  0.562220       False  
        2016-12-01 00:00:00+00:00  0.562220       False  
...                                     ...         ...  
49751   2016-04-01 00:00:00+00:00  0.487635       False  
        2016-05-01 00:00:00+00:00  0.487635        True  
49782   2016-08-01 00:00:00+00:00  0.644900        True  
49908   2016-03-01 00:00:00+00:00  0.542775        True  
49934   2016-05-01 00:00:00+00:00  0.019736        True  

[198115 rows x 6 columns]

In [8]:
usr_id = 117
df.loc[usr_id]

,f_0,f_1,f_2,f_3,f_4,churned_3m
date,,,,,,
2016-08-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2016-09-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2016-10-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2016-11-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2016-12-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2017-01-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2017-02-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2017-03-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False
2017-04-01 00:00:00+00:00,0.185834,0.257087,0.681947,0.343844,0.56222,False


In [9]:
monthly_payments.loc[lambda row: row['user_id'] == usr_id].sort_values("date")

,user_id,date,payment
49624,117,2016-08-01 00:00:00+00:00,1
179485,117,2016-09-01 00:00:00+00:00,1
30483,117,2016-10-01 00:00:00+00:00,1
163337,117,2016-11-01 00:00:00+00:00,1
5412,117,2016-12-01 00:00:00+00:00,1
125695,117,2017-01-01 00:00:00+00:00,1
61818,117,2017-02-01 00:00:00+00:00,1
147600,117,2017-03-01 00:00:00+00:00,1
72659,117,2017-04-01 00:00:00+00:00,1
116737,117,2017-05-01 00:00:00+00:00,1


In [37]:
df.loc[df.sample(1).index[0][0]]

,f_0,f_1,f_2,f_3,f_4,churned_3m
date,,,,,,
2016-12-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-01-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-02-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-03-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-04-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-05-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-06-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-07-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False
2017-08-01 00:00:00+00:00,0.157628,0.623363,0.639741,0.222977,0.357189,False


Since last observed is 2018-01-01 we assume that all observations with date 2017-11-01, 2017-12-01, 2018-01-01 are censored. Instead of fitting a survival analysis model, we could simply remove these dates

In [9]:
training_data = user_table.merge(df[['user_id', 'date', 'churned_3m']], on='user_id', how='left').set_index(['user_id', 'date'])
validation_data = training_data.loc[training_data.index.get_level_values(1).to_series().between('2017-07-01', '2017-10-01').values]
training_data = training_data.loc[~training_data.index.get_level_values(1).to_series().between('2017-07-01', '2018-01-01').values]
training_data

f_0       f_1       f_2  \
user_id date                                                           
117     2016-08-01 00:00:00.000000 UTC  0.185834  0.257087  0.681947   
        2016-09-01 00:00:00.000000 UTC  0.185834  0.257087  0.681947   
        2016-10-01 00:00:00.000000 UTC  0.185834  0.257087  0.681947   
        2016-11-01 00:00:00.000000 UTC  0.185834  0.257087  0.681947   
        2016-12-01 00:00:00.000000 UTC  0.185834  0.257087  0.681947   
...                                          ...       ...       ...   
49751   2016-04-01 00:00:00.000000 UTC  0.569852  0.289845  0.461525   
        2016-05-01 00:00:00.000000 UTC  0.569852  0.289845  0.461525   
49782   2016-08-01 00:00:00.000000 UTC  0.540032  0.502296  0.185834   
49908   2016-03-01 00:00:00.000000 UTC  0.420014  0.449307  0.200692   
49934   2016-05-01 00:00:00.000000 UTC  0.636215  0.440063  0.393347   

                                             f_3       f_4  churned_3m  
user_id date                                                            
117     2016-08-01 00:00:00.000000 UTC  0.343844  0.562220       False  
        2016-09-01 00:00:00.000000 UTC  0.343844  0.562220       False  
        2016-10-01 00:00:00.000000 UTC  0.343844  0.562220       False  
        2016-11-01 00:00:00.000000 UTC  0.343844  0.562220       False  
        2016-12-01 00:00:00.000000 UTC  0.343844  0.562220       False  
...                                          ...       ...         ...  
49751   2016-04-01 00:00:00.000000 UTC  0.374787  0.487635       False  
        2016-05-01 00:00:00.000000 UTC  0.374787  0.487635        True  
49782   2016-08-01 00:00:00.000000 UTC  0.075061  0.644900        True  
49908   2016-03-01 00:00:00.000000 UTC  0.535565  0.542775        True  
49934   2016-05-01 00:00:00.000000 UTC  0.496451  0.019736        True  

[171335 rows x 6 columns]

Note that there are no time varying covariates and we assume that the features are already standardized

In [10]:
X = training_data.drop(columns='churned_3m')
y = training_data['churned_3m']

In [11]:
y.value_counts()

churned_3m
False    123262
True      48073
Name: count, dtype: int64

We have imbalanced dataset and should be handled, however may be out of scope

In [12]:
from sklearn import metrics
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, r2_score, precision_score, recall_score, mean_squared_error
lr = XGBClassifier()
kfold = KFold(n_splits=5, random_state=9565687, shuffle=True)
splits = kfold.split(X, y)
for idx_train, idx_test in [next(splits)]:
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_test, y_test = X.iloc[idx_test], y.iloc[idx_test]
    lr.fit(X_train, y_train)
    y_score = lr.predict_proba(X_test)[:,1]
    y_pred = lr.predict(X_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_score)
    print({'mse': float(mean_squared_error(y_test, y_pred)),
           'r2': r2_score(y_test, y_score),
           'acc': float(accuracy_score(y_test, y_score >= 0.5)), 
           'auc': float(metrics.auc(fpr, tpr)),
           'precision': float(precision_score(y_test, y_pred)),
           'recall': float(recall_score(y_test, y_pred)),})

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/wiz/Library/Caches/pypoetry/virtualenvs/churn-adFSSGOr-py3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <89AD948E-E564-3266-867D-7AF89D6488F0> /Users/wiz/Library/Caches/pypoetry/virtualenvs/churn-adFSSGOr-py3.13/lib/python3.13/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), 

AUC is a good metric as we have an unbalanced dataset and mse and r2 are classical metrics for goodness of fit, however have drawbacks and should only be considered as indicators

In [ ]:
X_val, y_val = validation_data.drop(columns='churned_3m'), validation_data.loc[:,'churned_3m']
y_score_val = lr.predict_proba(X_val)[:,1]
y_pred_val = lr.predict(X_val)
fpr, tpr, _ = metrics.roc_curve(y_val, y_score_val)
print({'mse': float(mean_squared_error(y_val, y_pred_val)),
        'r2': r2_score(y_val, y_score_val),
        'acc': float(accuracy_score(y_val, y_pred_val)), 
        'auc': float(metrics.auc(fpr, tpr)),
        'precision': float(precision_score(y_val, y_pred_val)),
        'recall': float(recall_score(y_val, y_pred_val)),})

{'mse': 0.07660803007922654, 'r2': -0.282930850982666, 'acc': 0.9233919699207734, 'auc': 0.5530443349375164, 'precision': 0.2, 'recall': 0.001762114537444934}


In [ ]:
pd.concat([y_val.reset_index(drop=True), pd.Series(y_pred_val, name='prediction')], axis=1).value_counts()

churned_3m  prediction
False       0             13751
True        0              1133
False       1                 8
True        1                 2
Name: count, dtype: int64

In [ ]:
y_val.value_counts(), y_val.index.get_level_values(1).value_counts()

(churned_3m
 False    13759
 True      1135
 Name: count, dtype: int64,
 date
 2017-07-01 00:00:00.000000 UTC    5366
 2017-08-01 00:00:00.000000 UTC    4954
 2017-09-01 00:00:00.000000 UTC    4574
 Name: count, dtype: int64)

The model is not good out-of-sample